In [1]:
!python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: 'c:\Users\melan\AppData\Local\Programs\Python\Python311\python.exe'
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... PASS

Compiling [2/10]: MatrixFactorization_Cython_Epoch.pyx... 
Compiling [2/10]: MatrixFactorization_Cython_Epoch.pyx... PASS

Compiling [3/10]: Compute_Similarity_Cython.pyx... 
Compiling [3/10]: Compute_Similarity_Cython.pyx... PASS

Compiling [4/10]: SLIM_BPR_Cython_Epoch.pyx... 
Compiling [4/10]: SLIM_BPR_Cython_Epoch.pyx... FAIL: Command 'c:\Users\melan\AppData\Local\Programs\Python\Python311\python.exe c:\Users\melan\shared-folder\RecSys\RecSys-Competition/CythonCompiler/compile_script.py SLIM_BPR_Cython_Epoch.pyx build_ext --inplace' returned non-zero exit status 1.

Compiling [5/10]: Sparse_Matrix_Tree_CSR.pyx... 
Compiling

C:\Users\melan\AppData\Local\Programs\Python\Python311\Lib\site-packages\Cython\Compiler\Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: c:\Users\melan\shared-folder\RecSys\RecSys-Competition\Recommenders\MatrixFactorization\Cython\MatrixFactorizationImpressions_Cython_Epoch.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
C:\Users\melan\AppData\Local\Programs\Python\Python311\Lib\site-packages\Cython\Compiler\Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: c:\Users\melan\shared-folder\RecSys\RecSys-Competition\Recommenders\MatrixFactorization\Cython\MatrixFactorization_Cython_Epoch.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
C:\Users\melan\AppData\Local\Programs\Python\Python311\Lib\site-packages\Cython\Compiler\Main.py:381: FutureWarning: Cython directive 'language_level' no

In [2]:
from Recommenders.Ensambles import LinearCombination
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

In [3]:
import pandas as pd
import matplotlib.pyplot as pyplot
import numpy as np

In [4]:
np.int = int
np.float = float 
np.bool = bool

## Import Dataset

In [5]:
URM_all_dataframe = pd.read_csv('data_train.csv', 
                                sep=",", 
                                header= 0, 
                                dtype={0:int, 1:int, 2:float},
                                engine='python')

URM_all_dataframe.columns = ["UserID", "ItemID", "Interaction"]

In [6]:
# Move to sparse format
import scipy.sparse as sps

URM_all = sps.coo_matrix((URM_all_dataframe["Interaction"].values, 
                          (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values))) ## .values --> numpy array, df[..] --> pd series

In [7]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2591 (19.9%) Users that have less than 1 test interactions


## Ensamble

In [8]:
# Initialize the recommenders for the ensamble
kNN_user = UserKNNCFRecommender(URM_train)
kNN_item = ItemKNNCFRecommender(URM_train)

rec_list_KNN = [kNN_user, kNN_item]

UserKNNCFRecommender: URM Detected 652 ( 5.0%) users with no interactions.
UserKNNCFRecommender: URM Detected 254 ( 1.1%) items with no interactions.
ItemKNNCFRecommender: URM Detected 652 ( 5.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 254 ( 1.1%) items with no interactions.


In [9]:
# Create hyperparameters list
hyperp_user = {
                'merge_topPop': 'False',
                'topPop_factor': 0,
                'similarity': 'asymmetric',
                'feature_weighting': 'none',
                'topK': 837,
                'shrink': 945,
                'normalize' : False,
                'URM_bias' : True,
                'asymmetric_alpha': 1.2314034837290184,
    
}

hyperp_item = {'merge_topPop': False, 
               'topPop_factor': 0,
               'similarity': 'asymmetric',
               'feature_weighting': 'TF-IDF',
               'topK': 8,
               'shrink': 676,
               'normalize' : False,
               'URM_bias' : True,
               'asymmetric_alpha': 0.13379550739836213,
}

hyperp_P3 = {'alpha': 0.3492670788553423,
             'beta': 0.19804626346610282,
             'topK': 22,
             'normalize_similarity': True}



hyperp_list_KNN = [hyperp_user, hyperp_item]
weights_KNN_list = [0.30255328514425894, 0.8216579334567079]


In [10]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
hyperp_SLIM = {'topK': 4857,
               'l1_ratio': 0.09561217053667839,
               'alpha': 0.0009223033510385621}


In [11]:
rec_list = [LinearCombination(URM_train,rec_list_KNN,hyperp_list_KNN, weights_KNN_list), 
            RP3betaRecommender(URM_train),
            SLIMElasticNetRecommender(URM_train)]

Linear_Combination_Ensamble_Recommender_Class: URM Detected 652 ( 5.0%) users with no interactions.
Linear_Combination_Ensamble_Recommender_Class: URM Detected 254 ( 1.1%) items with no interactions.
RP3betaRecommender: URM Detected 652 ( 5.0%) users with no interactions.
RP3betaRecommender: URM Detected 254 ( 1.1%) items with no interactions.
SLIMElasticNetRecommender: URM Detected 652 ( 5.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 254 ( 1.1%) items with no interactions.


In [12]:
hyperp_list = [{'merge_topPop': True, 'topPop_factor': 0.04286958780262326,}, hyperp_P3, hyperp_SLIM]

In [13]:
# Instantiate the Ensamble
# If you don't specify the weights, the contribute of each Recommender will be the same.
weights_list_ens = [0.0, 0.46494854216480586, 0.5203670395256292]
Ensamble = LinearCombination(URM_train, recommenders_list= rec_list, hyperparameters_dicts_list= hyperp_list)

Linear_Combination_Ensamble_Recommender_Class: URM Detected 652 ( 5.0%) users with no interactions.
Linear_Combination_Ensamble_Recommender_Class: URM Detected 254 ( 1.1%) items with no interactions.


In [14]:
# Fit the models in the ensamble
Ensamble.fit(merge_topPop= True, topPop_factor= 2.0080060309776533e-08)

Similarity column 13025 (100.0%), 8478.41 column/sec. Elapsed time 1.54 sec
Successfully fitted Recommender 1 : UserKNNCFRecommender
Similarity column 22348 (100.0%), 9344.84 column/sec. Elapsed time 2.39 sec
Successfully fitted Recommender 2 : ItemKNNCFRecommender
Successfully fitted Recommender 1 : Linear_Combination_Ensamble_Recommender_Class
RP3betaRecommender: Similarity column 22348 (100.0%), 4923.79 column/sec. Elapsed time 4.54 sec
Successfully fitted Recommender 2 : RP3betaRecommender
SLIMElasticNetRecommender: Processed 11358 (50.8%) in 5.00 min. Items per second: 37.86
SLIMElasticNetRecommender: Processed 22348 (100.0%) in 9.42 min. Items per second: 39.55
Successfully fitted Recommender 3 : SLIMElasticNetRecommender


## Pipeline with Ensamble tuned precision@10

In [15]:
from Recommenders.Ensambles import PipelineStep

In [16]:
pipStep = PipelineStep(URM_train, Ensamble, n_relevant_per_user= 10)

Pipeline_Step_Ensamble_Recommender_Class: URM Detected 652 ( 5.0%) users with no interactions.
Pipeline_Step_Ensamble_Recommender_Class: URM Detected 254 ( 1.1%) items with no interactions.


In [17]:
output_URM_train = pipStep.compute_output_URM(remove_non_relevant_items= True, n_relevant_items_per_user= 10)
output_URM_train

MemoryError: Unable to allocate 6.51 GiB for an array with shape (3, 13025, 22348) and data type float64

In [ ]:
initial_users, initial_items = URM_train.shape
final_users, final_items = output_URM_train.shape

print("n_users\tinput: ", initial_users, "\toutput: ", final_users)
print("n_items\tinput: ", initial_items, "\toutput: ", final_items)
print("reduction factor for items: ", (initial_items-final_items)/initial_items)

n_users	input:  13025 	output:  13025
n_items	input:  22348 	output:  18278
reduction factor for items:  0.18211920529801323


In [ ]:
# Save URM_train, output_URM_train and URM_validation for consistency
from scipy.sparse import csr_matrix, save_npz
save_npz('URM_train.npz', URM_train)
save_npz('output_URM_train.npz', output_URM_train)
save_npz('URM_validation.npz', URM_validation)

## Test training on URM_output vs URM_input

In [ ]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

In [ ]:
hyperp_MAP = {'alpha': !!!, 'beta': !!!, 'topK': !!!, 'normalize_similarity': !!!} # MAP_MIN_DEN@10